CONTROLE DE AVISOS

In [ ]:
import warnings
#Essa linha serve para ignorar os avisos que aparecem no código
warnings.filterwarnings("ignore")

INSTALAÇÃO DAS BIBLIOTECAS NECESSÁRIAS PARA A APLICAÇÃO

In [ ]:
!python -m pip install crewai

IMPORTAÇÃO DAS BIBLIOTECAS NECESSÁRIAS E DEFINIÇÃO DE VARIÁVEIS DE AMBIENTE

In [15]:
from crewai import Agent, Task, Crew
import os 
from IPython.display import Markdown

#Definindo a chave de API do OpenAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
#Definindo o modelo do ChatGPT que será usado
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

CRIAÇÃO DOS AGENTES

In [ ]:
#Criação do Agente por meio de uma instanciação da classe Agent
planner = Agent(
    #Aqui é definido o nome/função do agente
    role="Content Planner",
    #Aqui é definido o objetivo do agente
    goal="Plan engaging and factually acurate content on {topic}",
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory="You're working on planning a blog article about the topic: {topic}."
    "You collect information that helps the audience learn something and make informed decisions."
    "Your work is the basis for the Content Writer to write an article on this topic",
    #Aqui é definido que ele não pode delegar tarefas
    allow_delegation=False,
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

In [ ]:
#Criação do Agente por meio de uma instanciação da classe Agent
writer = Agent(
    #Aqui é definido o nome/função do agente
    role="Content Writer",
    #Aqui é definido o objetivo do agente
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory="You're working on a writing a new opinion piece about the topic: {topic}."
    "You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic."
    "You follow the main objectives and direction of the outline, as provide by the Content Planner."
    "You also provide objective and impartial insights and back them up with information provide by the Content Planner."
    "You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.",
    #Aqui é definido que ele não pode delegar tarefas
    allow_delegation=False,
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

In [ ]:
#Criação do Agente por meio de uma instanciação da classe Agent
editor = Agent(
    #Aqui é definido o nome/função do agente
    role="Editor",
    #Aqui é definido o objetivo do agente
    goal="Edit a given blog post to align with the writing style of the organization.",
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory="You are an editor who receives a blog post from the Content Writer."
    "Your goal is to review the blog post to ensure that it follows journalistic"
    "best practices, provides balanced viewpoints when providing opinions or assertions,"
    "and also avoids major controversial topics or opinions when possible.",
    #Aqui é definido que ele não pode delegar tarefas
    allow_delegation=False,
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

CRIAÇÃO DAS TASKS

In [5]:
#Criação da task de planejamento do artigo
plan = Task(
    #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, a tarefa planejada
    #Nesse caso, planejar um artigo sobre um determinado tópico
    description=(
        "1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    #Aqui é definido a saída esperada, o que esperamos que o agente entregue
    expected_output="A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.",
    #Aqui é definido o Agente responsável pela tarefa, que, 
    #nesse caso, será o que irá planejar o conteúdo, o Planner
    agent=planner
)

In [6]:
#Criação da task de escrita do artigo
write = Task(
    #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, a tarefa planejada
    #Nesse caso, escrever um artigo sobre um determinado tópico
    description=(
        "1. Use the content plan to craft a compelling blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named in an engaging manner.\n"
        "4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and alignment with the brand's voice.\n"
    ),
    #Aqui é definido a saída esperada, o que esperamos que o agente entregue
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    #Aqui é definido o Agente responsável pela tarefa, que, 
    #nesse caso, será o que irá escrever o conteúdo do artigo, o Writer
    agent=writer
)

In [7]:
#Criação da task de edição do artigo
edit = Task(
    #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, a tarefa planejada
    #Nesse caso, editar um artigo sobre um determinado tópico
    description=("Proofread the given blog post for grammatical errors and alignment with the brand's voice."),
    #Aqui é definido a saída esperada, o que esperamos que o agente entregue
    #Nesse caso, um artigo pronto para ser publicado
    expected_output="A well-written blog post in markdown format, ready for publication. each section should have 2 or 3 paragraphs.",
    #Aqui é definido o Agente responsável pela tarefa, que,
    #nesse caso, será o que irá editar o conteúdo do artigo, o Editor
    agent=editor
)

CRIAÇÃO DO CREW (GRUPO DOS AGENTES QUE TRABALHAM JUNTOS)

In [ ]:
crew = Crew(
    #Definição da lista dos agentes que irão participar do processo
    agents=[planner, writer, editor],
    #Definição da lista das tarefas que irão participar, 
    #as tarefas serão realizadas na sequência que estão nessa lista
    #ou seja, primeiro o plan, depois o write e por fim o edit
    tasks=[plan, write, edit],
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

CHAMANDO A FUNÇÃO CREW PARA QUE O SISTEMA SEJA REALIZADO

In [13]:
# Iniciar o processo de criação do artigo com o tema "Inteligência Artificial"
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 
[2025-03-12 07:32:36][🚀 CREW 'CREW' STARTED, 7A93C07F-A272-4291-95B3-DC3AFD3B1C85]: 2025-03-12 07:32:36.666543
 
[2025-03-12 07:32:36][📋 TASK STARTED: 1. PRIORITIZE THE LATEST TRENDS, KEY PLAYERS, AND NOTEWORTHY NEWS ON ARTIFICIAL INTELLIGENCE.
2. IDENTIFY THE TARGET AUDIENCE, CONSIDERING THEIR INTERESTS AND PAIN POINTS.
3. DEVELOP A DETAILED CONTENT OUTLINE INCLUDING AN INTRODUCTION, KEY POINTS, AND A CALL TO ACTION.
4. INCLUDE SEO KEYWORDS AND RELEVANT DATA OR SOURCES.]: 2025-03-12 07:32:36.684632
 
[2025-03-12 07:32:36][🤖 AGENT 'CONTENT PLANNER' STARTED TASK]: 2025-03-12 07:32:36.694230
# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.
 
[2025-03-12 07:32:36][🤖 LLM CALL STARTED

VISUALIZAÇÃO DO ARTIGO "FORMATADO"

In [ ]:
Markdown(result.raw)

# Understanding Artificial Intelligence: Latest Trends and Key Players

## I. Introduction

In today's rapidly evolving technological landscape, Artificial Intelligence (AI) stands at the forefront of innovation. AI, a concept dating back to the 1950s, has now become a pivotal force driving advancements across various sectors. Its ability to simulate human intelligence and perform tasks traditionally requiring human intervention has revolutionized industries worldwide. The importance of AI in modern technology cannot be overstated. From enhancing efficiency in business operations to enabling breakthroughs in healthcare and finance, AI has cemented its position as a game-changer in the digital age.

## II. Latest Trends in Artificial Intelligence

### A. Advancements in machine learning algorithms
One of the most significant trends in AI is the continuous evolution of machine learning algorithms. Fueled by vast amounts of data, these algorithms have enabled AI systems to learn, adapt, and improve their performance autonomously. This trend has led to unparalleled accuracy in tasks like image recognition, natural language processing, and predictive analytics.

### B. Rise in the use of AI in healthcare and finance sectors
AI's impact on healthcare and finance has been transformative. In healthcare, AI-powered tools are revolutionizing diagnostics, personalized treatment plans, and drug development. Similarly, in the finance sector, AI is optimizing risk management, fraud detection, and customer service, ultimately reshaping the way financial institutions operate.

### C. Ethical considerations in AI development
As AI continues to advance, ethical considerations have come to the forefront. Issues such as bias in algorithms, data privacy, and accountability in AI decision-making processes are critical factors that must be carefully addressed to ensure responsible AI innovation.

## III. Key Players in Artificial Intelligence

### A. Google Brain and DeepMind
Google Brain and DeepMind, subsidiaries of Alphabet Inc., are at the forefront of AI research and development. Their work in areas such as deep learning, reinforcement learning, and neural networks has paved the way for groundbreaking AI applications.

### B. IBM Watson
IBM Watson, IBM's AI platform, has made significant contributions to AI in areas like natural language processing, data analytics, and cognitive computing. Its integration into various industries has demonstrated the vast potential of AI in driving innovation and efficiency.

### C. Tesla and SpaceX with Elon Musk
Elon Musk's companies, Tesla and SpaceX, have also made notable strides in AI through the development of autonomous vehicles, robotic systems, and space exploration technologies. Musk's vision for AI extends beyond Earth, aiming to push the boundaries of human-AI collaboration in the quest for innovation.

## IV. Noteworthy News in the AI Industry

### A. Recent breakthroughs in natural language processing
Advancements in natural language processing have unlocked new possibilities in AI interaction, enabling more natural and intuitive communication between humans and machines. This progress is reshaping the way we interact with AI-powered systems, from virtual assistants to language translation tools.

### B. Adoption of AI in autonomous vehicles
The adoption of AI in autonomous vehicles has gained significant traction, with major automotive companies investing in AI-driven technologies for self-driving cars. The integration of AI in transportation promises increased safety and efficiency, foretelling a future where autonomous vehicles are commonplace on the roads.

## V. Target Audience Analysis

### A. Professionals in the tech industry seeking updates on AI
For tech professionals, staying informed on the latest AI trends is crucial for competitiveness in a rapidly evolving industry. Insights into key players and emerging technologies provide valuable knowledge for career advancement and innovation.

### B. Businesses looking to integrate AI solutions into their operations
Businesses exploring AI solutions can benefit from understanding current trends and key players in the industry. Adopting AI strategically can drive growth, optimize processes, and enhance customer experiences in today's data-driven economy.

### C. Students and researchers interested in AI developments
Students and researchers exploring the realms of AI can gain valuable insights from tracking industry trends and advancements. Access to the latest news and key players in AI can inspire research opportunities, educational pursuits, and impactful contributions to the field.

## VI. Call to Action

### A. Encourage readers to stay informed on AI trends by subscribing to industry newsletters
To stay updated on the dynamic AI landscape, readers are encouraged to subscribe to industry newsletters and publications. By receiving curated insights and news updates, individuals can deepen their understanding of AI developments and remain at the forefront of innovation.

### B. Prompt engagement by sharing thoughts or questions on social media platforms
Engagement is crucial to fostering a vibrant AI community. Readers are invited to share their thoughts, questions, and experiences related to AI on social media platforms. By engaging in dialogue and knowledge-sharing, individuals can contribute to a collaborative ecosystem focused on advancing AI technologies.

In conclusion, understanding the latest trends and key players in Artificial Intelligence is essential for navigating the complex and dynamic AI landscape. As AI continues to shape industries and redefine the way we interact with technology, staying informed and engaging with the AI community are critical steps towards harnessing the full potential of AI-driven innovation. Let's embrace the opportunities AI presents and embark on a journey of exploration and discovery in the realm of Artificial Intelligence.

DESENVOLVIMENTO DE UM ARTIGO FEITO PELOS AGENTES COM MEU TEMA

In [16]:
topic = "Soccer"
result = crew.kickoff(inputs={"topic": topic})

 
[2025-03-12 07:35:29][🚀 CREW 'CREW' STARTED, 7A93C07F-A272-4291-95B3-DC3AFD3B1C85]: 2025-03-12 07:35:29.401806
 
[2025-03-12 07:35:29][📋 TASK STARTED: 1. PRIORITIZE THE LATEST TRENDS, KEY PLAYERS, AND NOTEWORTHY NEWS ON SOCCER.
2. IDENTIFY THE TARGET AUDIENCE, CONSIDERING THEIR INTERESTS AND PAIN POINTS.
3. DEVELOP A DETAILED CONTENT OUTLINE INCLUDING AN INTRODUCTION, KEY POINTS, AND A CALL TO ACTION.
4. INCLUDE SEO KEYWORDS AND RELEVANT DATA OR SOURCES.]: 2025-03-12 07:35:29.420954
 
[2025-03-12 07:35:29][🤖 AGENT 'CONTENT PLANNER' STARTED TASK]: 2025-03-12 07:35:29.420954
# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Soccer.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.
 
[2025-03-12 07:35:29][🤖 LLM CALL STARTED]: 2025-03-12 07:35:29.420954
 
[2

In [17]:
Markdown(result.raw)

# The Thrilling World of Soccer

## Introduction

Soccer, known as football in many parts of the world, has firmly established itself as the most popular sport globally. Its universal appeal transcends borders and cultures, uniting fans worldwide in their passion for the beautiful game. As we delve into the latest trends and key players in the soccer realm, we uncover a dynamic landscape that continues to captivate audiences with its exhilarating matches and unforgettable moments.

## Latest Trends in Soccer

### VAR Technology Revolutionizing Matches
One of the most significant advancements in soccer in recent years is the introduction of Video Assistant Referee (VAR) technology. This innovation has revolutionized the sport by ensuring more accurate decision-making during matches. While still sparking debates among fans and pundits, VAR undeniably adds a new dimension to the game's fairness and integrity.

### Impact of COVID-19 on Soccer
The global pandemic brought about unprecedented challenges for soccer events and schedules. With matches being played behind closed doors and disruptions to leagues and tournaments, the soccer industry had to adapt quickly to ensure the safety of players, staff, and fans. Despite the setbacks, the resilience of the soccer community shone through, demonstrating the sport's ability to overcome adversity.

### Rise of Youth Academies Nurturing Talent
In recent years, there has been a noticeable emphasis on developing young talent through youth academies. Clubs are increasingly investing in their youth systems to groom the next generation of soccer stars. This trend not only fosters talent development but also promotes sustainability within clubs by cultivating a pipeline of skilled and dedicated players.

## Key Players in Soccer

### Lionel Messi: A Legend's Legacy
Lionel Messi stands out as one of the greatest players in soccer history, with a career marked by numerous achievements and accolades. His remarkable skill on the field, vision, and goal-scoring prowess have solidified his status as an iconic figure in the sport, inspiring generations of players and fans alike.

### Cristiano Ronaldo: A Global Phenomenon
Often compared to Messi, Cristiano Ronaldo brings his own brand of excellence to the soccer world. Known for his athleticism, work ethic, and goal-scoring abilities, Ronaldo commands a massive global following and remains a dominant force in the sport. The rivalry between Messi and Ronaldo has elevated both players to legendary status.

### Kylian Mbappé: The Future of Soccer
As a rising star in the soccer landscape, Kylian Mbappé has captured the attention of fans with his electrifying speed, technical skills, and maturity on the field. At a young age, Mbappé has already achieved remarkable success and is poised to become a key figure in the future of soccer, carrying on the legacy of great players before him.

## Noteworthy News in Soccer

### Transfer Updates and Major Tournaments
The soccer world is constantly abuzz with transfer speculations involving top clubs and players. These moves not only impact the competitive landscape of the sport but also generate excitement among fans eager to see new talent in action. Additionally, major tournaments like the FIFA World Cup and UEFA Champions League provide thrilling moments that captivate audiences worldwide.

### Managerial Changes and Team Dynamics
Beyond player transfers, managerial changes in prominent soccer teams can significantly influence a club's performance and strategies. The decisions made by managers shape the direction of teams and reflect the evolving nature of soccer tactics and leadership. Keeping an eye on these developments adds another layer of intrigue to the sport.

## Target Audience Analysis

Soccer enthusiasts, aspiring players, and sports journalists form a diverse yet interconnected audience that shares a deep passion for the sport. By catering to their interests and providing engaging content that resonates with their love for soccer, we can cultivate a community of dedicated fans and supporters who contribute to the ongoing conversation surrounding the game.

## Call to Action

As we continue to explore the ever-evolving world of soccer, I invite all readers to share their favorite soccer moments and engage with the content by providing their insights and perspectives. Stay connected for future updates on soccer news, player profiles, and exclusive insights into the world of sports. Let's come together to celebrate the spirit of soccer and the boundless excitement it brings to fans around the globe.

In conclusion, soccer remains a vibrant and thrilling sport that captivates audiences worldwide with its rich history, talented players, and exciting developments. As we embrace the latest trends, celebrate iconic players, and stay informed on noteworthy news, let's unite in our shared love for the beautiful game and look forward to the countless moments of magic that soccer has yet to offer.